<a href="https://colab.research.google.com/github/gb999/LuaLuaLua/blob/getting-data/LuaLuaLua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting the data

To download competition data from Kaggle the rules of the competition have to be accepted [here](https://www.kaggle.com/competitions/nlp-getting-started/data), by joining the competition.

After that, an API key has to be obtained.
Follow these [instructions](https://www.kaggle.com/docs/api) to do so.

Create a new secret named `kaggle.json` in Google Colab, copy the contents of the downloaded file into the value field. Enable Notebook access, and you are ready to go.





In [19]:
from google.colab import userdata
import json, os
kaggle_path = '/root/.config/kaggle'
kaggle_json = userdata.get('kaggle.json')
os.makedirs(kaggle_path, exist_ok=True)

with open(kaggle_path + '/kaggle.json', 'w') as f:
    f.write(kaggle_json)

os.chmod(kaggle_path + '/kaggle.json', 0o600)

In [22]:
!pip install kaggle
!kaggle competitions download -c nlp-getting-started -q
!unzip nlp-getting-started.zip

Archive:  nlp-getting-started.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# TODO


In [29]:
import pandas as pd
# Load DataFrames
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(train_df.shape)
print(test_df.shape)
print(train_df.iloc[40])

(7613, 5)
(3263, 4)
id                                                         59
keyword                                                ablaze
location                                       Live On Webcam
text        Check these out: http://t.co/rOI2NSmEJJ http:/...
target                                                      0
Name: 40, dtype: object
